In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

/Users/jeremyben-meir/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/Users/jeremyben-meir/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


# OPENDATA: neighborhood coef on sale price

In [33]:
sales = pd.read_csv("NYC_Citywide_Rolling_Calendar_Sales.csv")
sales.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY',
       'TAX CLASS AT PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING CLASS AT PRESENT', 'ADDRESS', 'APARTMENT NUMBER', 'ZIP CODE',
       'RESIDENTIAL UNITS', 'COMMERCIAL UNITS', 'TOTAL UNITS',
       'LAND SQUARE FEET', 'GROSS SQUARE FEET', 'YEAR BUILT',
       'TAX CLASS AT TIME OF SALE', 'BUILDING CLASS AT TIME OF SALE',
       'SALE PRICE', 'SALE DATE'],
      dtype='object')

In [34]:
sales['NEIGHBORHOOD'] = sales['NEIGHBORHOOD'].apply(lambda n: n.replace(" (79-96)","").replace(" (59-79)",""))

sales = sales[~sales['NEIGHBORHOOD'].isna()]
sales = sales[~sales['TOTAL UNITS'].isna()]
sales = sales[~sales['LAND SQUARE FEET'].isna()]
sales = sales[~sales['GROSS SQUARE FEET'].isna()]
sales = sales[~sales['YEAR BUILT'].isna()]
sales = sales[~sales['TAX CLASS AT TIME OF SALE'].isna()]
sales = sales[~sales['BUILDING CLASS AT TIME OF SALE'].isna()]
sales['SALE PRICE'] = sales['SALE PRICE'].str.replace(",","").astype(int)
sales = sales[~((sales['SALE PRICE'].isna())|(sales['SALE PRICE']<=0))]
sales["PRICE/UNIT"] = sales['SALE PRICE']/sales['TOTAL UNITS']

X = pd.concat([
        pd.get_dummies(sales['NEIGHBORHOOD'], prefix="neighborhood", drop_first=True),
        sales['TOTAL UNITS'].astype(int),
        sales['LAND SQUARE FEET'].str.replace(",","").astype(int),
        sales['GROSS SQUARE FEET'].astype(int),
        sales['YEAR BUILT'].astype(int),
        pd.get_dummies(sales['TAX CLASS AT TIME OF SALE'], prefix="taxclass", drop_first=True),
        pd.get_dummies(sales['BUILDING CLASS AT TIME OF SALE'], prefix="bldgclass", drop_first=True),
    ],axis=1)

Y = sales['SALE PRICE']
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
model.summary()

neighborhood_params = model.params.loc[[val for val in model.params.index if "neighborhood_" in val]]
significant_params = neighborhood_params[neighborhood_params.index.isin(model.pvalues[model.pvalues<=.05].index)]
# neighborhood_params = significant_params.loc[[model.params.index]]
significant_params.sort_values()



/Users/jeremyben-meir/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


neighborhood_DOWNTOWN-FULTON MALL        -3.163155e+07
neighborhood_JAVITS CENTER               -1.698351e+07
neighborhood_FLATIRON                    -1.651043e+07
neighborhood_EAST VILLAGE                 7.524639e+06
neighborhood_HUNTS POINT                  8.567422e+06
neighborhood_WILLIAMSBURG-NORTH           8.661093e+06
neighborhood_LOWER EAST SIDE              8.770358e+06
neighborhood_CLINTON                      9.242175e+06
neighborhood_TRIBECA                      9.265173e+06
neighborhood_GREENWICH VILLAGE-WEST       9.900629e+06
neighborhood_DOWNTOWN-METROTECH           1.003332e+07
neighborhood_CHELSEA                      1.005147e+07
neighborhood_UPPER WEST SIDE              1.069827e+07
neighborhood_BROOKLYN HEIGHTS             1.310206e+07
neighborhood_GREENWICH VILLAGE-CENTRAL    1.471068e+07
neighborhood_FASHION                      1.484955e+07
neighborhood_UPPER EAST SIDE              1.534424e+07
neighborhood_FINANCIAL                    2.412859e+07
neighborho

# AIRBNB: neighborhood coef on nightly price

In [35]:
listings = pd.read_csv("listings.csv")
listings.columns


/var/folders/__/xdj9wy117vzd8wjwwhpp9sph0000gn/T/ipykernel_19794/3649005725.py:1: DtypeWarning: Columns (67) have mixed types. Specify dtype option on import or set low_memory=False.
  listings = pd.read_csv("listings.csv")


Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_upd

In [36]:
listings['neighbourhood_cleansed'] = listings['neighbourhood_cleansed'].str.upper().str.replace(" DISTRICT","")
listings["price"] = listings['price'].str.replace("$","").str.replace(",","").astype(float)

listings = listings[~listings['neighbourhood_cleansed'].isna()]
listings = listings[~listings['number_of_reviews'].isna()]
listings = listings[~listings['bedrooms'].isna()]
listings = listings[~listings['beds'].isna()]
listings = listings[~listings['host_identity_verified'].isna()]
listings = listings[~listings['host_has_profile_pic'].isna()]
listings = listings[~listings['room_type'].isna()]
listings = listings[~listings['review_scores_rating'].isna()]
listings = listings[~listings['minimum_nights'].isna()]
listings = listings[~listings['host_is_superhost'].isna()]
listings = listings[listings['price']>0]

X = pd.concat([
        pd.get_dummies(listings['neighbourhood_cleansed'], prefix="neighborhood", drop_first=True),
        listings['number_of_reviews'],
        listings['bedrooms'],
        listings['beds'],
        pd.get_dummies(listings['host_is_superhost'], prefix="host_is_superhost", drop_first=True),
        pd.get_dummies(listings['host_has_profile_pic'], prefix="host_has_profile_pic", drop_first=True),
        pd.get_dummies(listings['host_identity_verified'], prefix="host_identity_verified", drop_first=True),
        pd.get_dummies(listings['room_type'], prefix="room_type", drop_first=True),
        listings['review_scores_rating'],
        listings['minimum_nights'],
    ],axis=1)

Y = listings['price']
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
model.summary()

neighborhood_params = model.params.loc[[val for val in model.params.index if "neighborhood_" in val]]
significant_params = neighborhood_params[neighborhood_params.index.isin(model.pvalues[model.pvalues<=.05].index)]
# neighborhood_params = significant_params.loc[[model.params.index]]
significant_params.sort_values(ascending=False)


/var/folders/__/xdj9wy117vzd8wjwwhpp9sph0000gn/T/ipykernel_19794/1629374173.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  listings["price"] = listings['price'].str.replace("$","").str.replace(",","").astype(float)
/Users/jeremyben-meir/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


neighborhood_TRIBECA              319.201512
neighborhood_FLATIRON             317.434750
neighborhood_JAMAICA ESTATES      306.806998
neighborhood_MIDTOWN              238.171332
neighborhood_SOHO                 211.993228
neighborhood_THEATER              183.252825
neighborhood_WEST VILLAGE         180.626892
neighborhood_MURRAY HILL          177.864433
neighborhood_GREENWICH VILLAGE    158.162670
neighborhood_CHELSEA              152.023399
neighborhood_FINANCIAL            127.144620
neighborhood_EAST VILLAGE         116.708387
neighborhood_HELL'S KITCHEN       114.932977
neighborhood_UPPER WEST SIDE      107.829249
neighborhood_GRAMERCY             104.208515
neighborhood_NOLITA                98.565600
neighborhood_LOWER EAST SIDE       97.259569
neighborhood_UPPER EAST SIDE       92.555050
dtype: float64

In [37]:
# n = "FLATIRON"
# print(sales.loc[sales['NEIGHBORHOOD']==n,"PRICE/UNIT"])
# print(sales.loc[sales['NEIGHBORHOOD']==n,"SALE PRICE"])
# print(sales.loc[sales['NEIGHBORHOOD']==n,"TOTAL UNITS"])
# expected_price = sales.loc[sales['NEIGHBORHOOD']==n,"PRICE/UNIT"].mean()
# expected_return = listings.loc[listings['neighbourhood_cleansed']==n,"price"].mean()*365
# print(expected_price)
# expected_return/expected_price

# Get compound sentiment of each review

In [7]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
reviews = pd.read_csv("reviews.csv")
reviews = reviews[~reviews["comments"].isna()]
# reviews["comments"] = reviews["comments"].astype(str)

[nltk_data] Downloading package vader_lexicon to /Users/jeremyben-
[nltk_data]     meir/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [8]:
#add sentiment to reviews

global count
count = 0
print(len(reviews["comments"]))

def add_score(comment):
    try:
        global count
        count+=1
        if count%100000 == 0:
            print(count)
        return sia.polarity_scores(comment)["compound"]
    except:
        print(comment)

reviews["sentiment"] = reviews["comments"].apply(lambda comment: add_score(comment))


927923
100000
200000
300000
400000
500000
600000
700000
800000
900000


In [30]:
# merge average sentiment into reviews

sentiment_df = reviews.groupby("listing_id").agg({"sentiment":"mean"})
listings_reviews = listings.merge(sentiment_df, how="inner", left_on='id', right_on='listing_id')


25149
927923
25147


In [51]:
reviews_summary = listings_reviews["review_scores_rating"].describe()
sentiment_summary = listings_reviews["sentiment"].describe()
print(reviews_summary)
print(sentiment_summary)

count    25147.000000
mean         4.612306
std          0.783162
min          0.000000
25%          4.600000
50%          4.830000
75%          5.000000
max          5.000000
Name: review_scores_rating, dtype: float64
count    25147.000000
mean         0.716129
std          0.239857
min         -0.998800
25%          0.646336
50%          0.768033
75%          0.865819
max          0.998700
Name: sentiment, dtype: float64


In [48]:
N=100000
np.clip(np.random.normal(reviews_summary["mean"],reviews_summary["std"],N),0,5)
np.clip(np.random.normal(sentiment_summary["mean"],sentiment_summary["std"],N),-1,1)

array([0.90078986, 0.2236116 , 0.67824891, ..., 0.6617221 , 0.670106  ,
       0.83942939])